In [1]:
%matplotlib inline
%load_ext autoreload

%autoreload 2
import tensorflow as tf
tf.enable_eager_execution()
tf.executing_eagerly()

True

In [2]:
!pip install ../forks/tensortrade_source/ -U

Processing c:\users\codeninja\dev\forks\tensortrade_source
  Created wheel for tensortrade: filename=tensortrade-0.0.2rc0-cp37-none-any.whl size=56841 sha256=10213f8412ef370143346d50c406ff97212431a9bda9d30724ad8a268b7c9793
  Stored in directory: C:\Users\CODENI~1\AppData\Local\Temp\pip-ephem-wheel-cache-ofyh2jgw\wheels\62\be\3a\90782c795f0ba93fec4a55c551ac7e8f7e7ea933797e4f71c0
Successfully built tensortrade
  Found existing installation: tensortrade 0.0.2rc0
    Uninstalling tensortrade-0.0.2rc0:
      Successfully uninstalled tensortrade-0.0.2rc0


In [3]:
import os
import sys
import warnings
import numpy
import ccxt
import pandas as pd
import numpy as np
from time import time
import math
import cProfile as cp
import matplotlib.pyplot as plt

from gym.spaces import Space, Box

config = './neat.config'

def warn(*args, **kwargs):
    pass

warnings.warn = warn
warnings.simplefilter(action='ignore', category=FutureWarning)
numpy.seterr(divide = 'ignore') 

sys.path.append(os.path.dirname(os.path.abspath('')))
sys.path.append(os.path.abspath('')+"\\tensortrade")


**Load Data**

In [4]:

data_file ='./data/processed/binance/btc_usdt_1h.csv' 
df = pd.read_csv(data_file, index_col=[0])


# number of days we want to pull from the dataframe 
days_of_data = 365

# number of data frames (our DF is in 1h timesteps)
frames = days_of_data * 24 * 5
# frames = len(df)
train_test_percentage = 0.4

x_train = int(frames * (1 - train_test_percentage))
x_test = int(frames - x_train)

df_train = df[-frames:(-x_test - 1)]
df_test = df[-x_test:]
print("Friend Shape", df.shape)
print('train shape', df_train.shape)
print('test shape', df_test.shape)
print('columns', df.columns)

Friend Shape (180236, 12)
train shape (26279, 12)
test shape (17520, 12)
columns Index(['open', 'high', 'low', 'close', 'volume', 'momentum_rsi', 'adx',
       'adx_long', 'adx_pos', 'adx_neg', 'trend_ema_fast', 'trend_ema_slow'],
      dtype='object')


**Train against training data**

In [ ]:
import tensortrade
from neat_trading_strategy import NeatTradingStrategy as TradingStrategy
from neat_reward_strategy import NeatRewardStrategy as ProfitStrategy
from tensortrade.actions import DiscreteActionStrategy as ActionStrategy
from tensortrade.exchanges.simulated import SimulatedExchange as Exchange
from tensortrade.features.scalers import MinMaxNormalizer
from tensortrade.features import FeaturePipeline
from tensortrade.environments import TradingEnvironment as Environment

print('fin imports')
normalize = MinMaxNormalizer(inplace=True)
feature_pipeline = FeaturePipeline(steps=[normalize])

reward_strategy = ProfitStrategy()
action_strategy = ActionStrategy(n_actions=20, instrument_symbol='BTC/USDT')
print('fin strats')

exchange = Exchange(data_frame=df_train,
                    pretransform = True,
                    base_instrument='USDT', 
                    commission_percent=0.75,
                    window_size=1,
                    max_allowed_slippage_percent=3.0,
                    min_order_amount=1E-4,
                    min_trade_amount=1E-4,
                    observation_columns = df_train.columns
                   )
print('fin exchange')

environment = Environment(exchange=exchange,
                                 action_strategy=action_strategy,
                                 reward_strategy=reward_strategy,
                                 feature_pipeline=feature_pipeline)
print('fin environment')
print('')

segments_in_day = 288
days = 1

strategy = TradingStrategy(environment=environment, 
                           neat_config=config, 
                           watch_genome_evaluation=True,
                           only_show_profitable=False,
                           learn_to_trade_theshold=-10000,
                           data_frame_window = segments_in_day * days
                          )

print("Running through ", strategy._data_frame_window, ' steps')
# cp.run("performance, winner, stats = strategy.run(generations=20)", 'evolution_stats')
performance, winner, stats = strategy.run(generations=200)

Population's average fitness: 867.65472 stdev: 1121.21564
Best fitness: 2239.10755 - size: (13, 169) - species 1 - id 1
Average adjusted fitness: 0.665
Mean genetic distance 3.676, standard deviation 0.749
Population of 200 members in 20 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    4    10   2239.1    0.619     4
     2    4    10   1764.5    0.464     1
     3    4    10    997.9    0.430     3
     4    4    10   2022.9    0.882     3
     5    4    10   2220.0    0.770     1
     6    4    10   2220.0    0.831     0
     7    4    10   1835.0    0.442     2
     8    4    10   2008.7    0.845     3
     9    4    10   2209.0    0.680     4
    10    4    10   2044.8    0.502     1
    11    4    10   2033.2    0.586     2
    12    4    10   2176.0    0.681     2
    13    4    10   2022.9    0.502     3
    14    4    10   2198.0    0.608     2
    15    4    10   1997.0    0.622     1
    16    4    10   2031.3    0.932    

**Evaluate against full dataset**

In [ ]:
exchange = Exchange(data_frame=df,
                    should_pretransform_obs = True,
                    base_instrument='USDT', 
                    timeframe='1h',
                    dtype=np.float32,
                    commission_percent=0.5,
                    window_size=1,
                    max_allowed_slippage_percent=3.0,
                    min_order_amount=1E-4,
                    min_trade_amount=1E-4
                   )
environment = Environment(exchange=exchange,
                                 action_strategy=action_strategy,
                                 reward_strategy=reward_strategy,
                                 feature_pipeline=feature_pipeline)

environment.reset()
from neat_trading_strategy import NeatTradingStrategy as TradingStrategy

strategy = TradingStrategy(environment=environment, watch_genome_evaluation=True, full_evaluation = True, neat_config=config)

In [ ]:
strategy.eval_genome(winner)

In [ ]:
print(environment.exchange.trades)
environment.exchange.performance.net_worth.plot()

In [ ]:
import visualize

visualize.plot_stats(stats, ylog=False, view=True)


In [ ]:
visualize.plot_species(stats, view=True)

In [ ]:
visualize.draw_net(strategy._config, winner, False, node_names=node_names)


**Save Strategy**

In [ ]:
columns = ["Action"]+list(exchange.data_frame.columns)

nodes = list(range(len(columns)*-1,1))
nodes.reverse()
node_names = dict( zip(nodes, columns))
print(node_names)

In [ ]:
dir(winner.connections.keys)
keys = list(winner.connections.values())

keys[0].weight/5

In [ ]:
from tensortrade.trades import Trade, TradeType
tt = TradeType(3)
t = Trade("btc/usdt",tt, 1, 100)
t.price
t.transact_price
t.fset('transact_price') = 123